In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# Step 1: Gather and Format Data

In [2]:
df = pd.read_csv("hungarian_nt_matches.csv")
df.head()

,away_goals,opponent,home_goals,home_or_away,opponent_goals,hungary_goals,outcome,date,match_type
0,1,Austria,4,H,1,4,W,1907-11-03,Friendly
1,3,Czech Republic,5,A,5,3,L,1907-10-06,Friendly
2,2,Czech Republic,5,H,2,5,W,1907-04-07,Friendly
3,3,Austria,5,H,3,5,W,1908-11-01,Friendly
4,7,England,0,H,7,0,L,1908-06-10,Friendly


Create binary column for wins vs. not-wins to easily get an average

In [3]:
df["is_win"] = df["outcome"] == "W"
df["is_win"] = df["is_win"].astype(int)
df.head()

,away_goals,opponent,home_goals,home_or_away,opponent_goals,hungary_goals,outcome,date,match_type,is_win
0,1,Austria,4,H,1,4,W,1907-11-03,Friendly,1
1,3,Czech Republic,5,A,5,3,L,1907-10-06,Friendly,0
2,2,Czech Republic,5,H,2,5,W,1907-04-07,Friendly,1
3,3,Austria,5,H,3,5,W,1908-11-01,Friendly,1
4,7,England,0,H,7,0,L,1908-06-10,Friendly,0


# Step 2: Get a list of countries for mapping

Need to merge some historic countries with existing ones

Original source of countries with codes + GDP: [https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv](https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv)

In [4]:
df_countries = pd.read_csv("country_gdp_data.csv")

countries = list(df_countries["COUNTRY"].str.strip())
countries

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas, The',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burma',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo, Democratic Republic of the',
 'Congo, Republic of the',
 'Cook Islands',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Curacao',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'Falkland Islands (Islas Malvinas)'

Look at countries that need fixing

In [5]:
wrong_countries = [c for c in df["opponent"].unique() if c not in countries]
sorted(wrong_countries)

['Asian All Stars',
 'Czechoslovakia',
 'East Germany',
 'England',
 'FYR Macedonia',
 'Netherlands East-Indies',
 'Northern Ireland',
 'Republic of Ireland',
 'Scotland',
 'South Korea',
 'Soviet Union',
 'USA',
 'United Arab Republic',
 'Wales',
 'West Germany',
 'Yugoslavia']

Fix them!

In [6]:
# first, keep a copy of the original opponent
df["original_opponent"] = df["opponent"]

# start with the easy ones
df.loc[df["opponent"] == "FYR Macedonia", "opponent"] = "Macedonia"
df.loc[df["opponent"] == "Republic of Ireland", "opponent"] = "Ireland"
df.loc[df["opponent"] == "South Korea", "opponent"] = "Korea, South"
df.loc[df["opponent"] == "USA", "opponent"] = "United States"

# some of them don't exist anymore or are not really countries
to_remove = ["Asian All Stars", "United Arab Republic", "Netherlands East-Indies"]
df.drop(df[df["opponent"].isin(to_remove)].index, axis=0, inplace=True)

# what about East and West Germany - let's combine them under Germany
df.loc[df["opponent"] == "East Germany", "opponent"] = "Germany"
df.loc[df["opponent"] == "West Germany", "opponent"] = "Germany"

# USSR = Russia
df.loc[df["opponent"] == "Soviet Union", "opponent"] = "Russia"

# Yugoslavia = Serbia
df.loc[df["opponent"] == "Yugoslavia", "opponent"] = "Serbia"

# Czechoslovakia = Czech Republic
df.loc[df["opponent"] == "Czechoslovakia", "opponent"] = "Czech Republic"

# and merge the countries of the UK
df.loc[df["opponent"].isin(["Scotland", "England", "Wales", "Northern Ireland"]), "opponent"] = "United Kingdom"

Check to see if we have any issues left

In [7]:
wrong_countries = [c for c in df["opponent"].unique() if c not in countries]
sorted(wrong_countries)

[]

# Step 2: Calculate Winning Percentages

Group by opponent and calculate winning %

In [8]:
opponent_records = df.groupby("opponent")["is_win"].agg({"win_percent": "mean", "total_games": "count"}).reset_index()
opponent_records["win_percent"] = round(opponent_records["win_percent"] * 100, 1)
opponent_records.sort_values(["win_percent", "total_games", "opponent"], ascending=[False, False, True])

,opponent,total_games,win_percent
47,Luxembourg,10,100.0
8,Azerbaijan,5,100.0
35,Iran,4,100.0
62,San Marino,4,100.0
2,Andorra,3,100.0
54,New Zealand,3,100.0
11,Bolivia,2,100.0
15,Canada,2,100.0
39,Japan,2,100.0
42,"Korea, South",2,100.0


*Optional: export records to CSV*

In [9]:
opponent_records.to_csv("opponent_records.csv", index=False)

Add dummy values for countries not in data

In [10]:
for c in countries:
    if c not in opponent_records["opponent"].values:
        opponent_records = opponent_records.append({
                                                    "opponent": c,
                                                    "total_games": 0,
                                                    "win_percent": -1
                                                   }, ignore_index=True)

Add country codes

In [11]:
df_countries.rename(columns={"COUNTRY": "opponent", "CODE": "country_code"}, inplace=True)
opponent_records = opponent_records.merge(df_countries[["opponent", "country_code"]], on="opponent", how="inner")
opponent_records.head()

,opponent,total_games,win_percent,country_code
0,Albania,6,83.3,ALB
1,Algeria,1,100.0,DZA
2,Andorra,3,100.0,AND
3,Antigua and Barbuda,1,100.0,ATG
4,Argentina,7,14.3,ARG


# Step 4: Use Plot.ly for better interactive map

In [12]:
import plotly.plotly as py

Create label text

In [13]:
def label_text(country):
    # check if we've ever played the country
    if country not in df["opponent"].unique():
        label = "{}: no games".format(country)
        return label
    # change some of the names to account for data merging
    country_label = country
    if country == "Russia":
        country_label = "Russia & USSR"
    elif country == "Germany":
        country_label = "Germany (East & West)"
    elif country == "Czech Republic":
        country_label = "Czech Republic & Czechoslovakia"
    elif country == "Serbia":
        country_label = "Serbia & Yugoslavia"
    
    # otherwise get stats and games
    win_percent = opponent_records.loc[opponent_records["opponent"] == country, "win_percent"].values[0]
    total_games = opponent_records.loc[opponent_records["opponent"] == country, "total_games"].values[0]
    # start with country and win percentage and number of games
    game_text = "games"
    if total_games == 1:
        game_text = "game"

    label = "{}: {}% ({} {})<br><br>".format(country_label, int(round(win_percent,0)), total_games, game_text)
    
    # if we have 10 games or fewer, loop through and list all games
    games = df[df["opponent"] == country]

    label += "Last 5 games: <br>"
    
    for g in games.sort_values("date", ascending=False).head(5).itertuples():
        if g.home_or_away == "H":
            label += "Hungary {}:{} {} ({})<br>".format(g.hungary_goals, g.opponent_goals, g.original_opponent, g.date)
        else:
            label += "{} {}:{} Hungary ({})<br>".format(g.original_opponent, g.opponent_goals, g.hungary_goals, g.date)

    return label

opponent_records["label"] = opponent_records.apply(lambda x: label_text(x["opponent"]), axis=1)
opponent_records.head()

,opponent,total_games,win_percent,country_code,label
0,Albania,6,83.3,ALB,Albania: 83% (6 games)<br><br>Last 5 games: <b...
1,Algeria,1,100.0,DZA,Algeria: 100% (1 game)<br><br>Last 5 games: <b...
2,Andorra,3,100.0,AND,Andorra: 100% (3 games)<br><br>Last 5 games: <...
3,Antigua and Barbuda,1,100.0,ATG,Antigua and Barbuda: 100% (1 game)<br><br>Last...
4,Argentina,7,14.3,ARG,Argentina: 14% (7 games)<br><br>Last 5 games: ...


Make the map!

In [14]:
data = [ dict(
        type = 'choropleth',
        locations = opponent_records['country_code'],
        z = opponent_records['win_percent'],
        text = opponent_records['label'],
        hoverinfo = "text",
        colorscale = [[0, "rgb(80, 80, 80)"], [0.01,"rgb(255,77,77)"],[0.25,"rgb(255,179,179)"],
                      [0.5,"rgb(255, 230, 230)"],[0.6,"rgb(204, 224, 255)"],
                      [0.7,"rgb(128, 179, 255)"],[1,"rgb(0,100,255)"]],
        zauto=True,
        autocolorscale = False,
        reversescale = False,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            #tickprefix = '%',
            title = 'Win percentage (%)'),
      ) ]

layout = dict(
    title = 'Worldwide win percentage of the Hungarian NT',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot(fig, validate=False, filename='worldwide-win-map')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~dasboth/0 or inside your plot.ly account where it is named 'worldwide-win-map'


Full screen version available here: [https://plot.ly/~dasboth/0.embed](https://plot.ly/~dasboth/0.embed)